In [1]:
## imporrt libarary 
import os
import pandas as pd
import numpy as np

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

In [2]:
# make data observation
new_test_data = pd.DataFrame({
    'longitude': [-118.31, -122.3],
    'latitude': [33.95, 37.9],
    'housing_median_age': [43.0, 30.0],
    'total_rooms': [1823.0, 1772.0],
    'total_bedrooms': [358.0, 471.0],
    'population': [1065.0, 880.0],
    'households': [342.0, 437.0],
    'median_income': [3.2708, 2.2672],
    'ocean_proximity': ['<1H OCEAN', 'NEAR BAY'],
    'rooms_per_household': [5.330409, 4.05492],
    'population_per_household': [3.114035, 2.01373]
})

In [3]:
new_test_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,population_per_household
0,-118.31,33.95,43.0,1823.0,358.0,1065.0,342.0,3.2708,<1H OCEAN,5.330409,3.114035
1,-122.30,37.90,30.0,1772.0,471.0,880.0,437.0,2.2672,NEAR BAY,4.054920,2.013730


In [4]:
#  set up authentication using services account 
# Authenticate using service account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/home/giansirait18/gian-023/sa-development.json"

In [5]:
# Load data from Google Biquery to local machine
project_id = 'dti-ds'
dataset_id = 'gian_dataset_023'
table_id = 'data_to_test'
region = 'us-central1'
bucket_name = 'gian_gcs_023'
blob_name = 'data/data_to_test.csv'

In [6]:
# Upload the model to Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob('model/Final_Model_Capstone.pkl')
    blob_model.upload_from_filename('Final_Model_Capstone.pkl')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

Uploading model succeeded


In [7]:
# load data from BQ
# using bigquery client 
# client = bigquery.Client(credentials=credentials,project=project_id)
client = bigquery.Client(project=project_id)

# query 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
df = query_job.result().to_dataframe()
df

/home/giansirait18/miniconda3/envs/california-new/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,int64_field_0,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,population_per_household
0,1656,-120.00,38.92,26.0,529.0,116.0,191.0,83.0,3.5000,INLAND,6.373494,2.301205
1,2057,-116.25,33.81,24.0,880.0,187.0,507.0,169.0,3.4583,INLAND,5.207101,3.000000
2,9295,-116.50,33.69,20.0,4810.0,1074.0,1304.0,740.0,2.2500,INLAND,6.500000,1.762162
3,7200,-120.50,38.87,10.0,81.0,41.0,55.0,16.0,4.9583,INLAND,5.062500,3.437500
4,9731,-120.50,37.32,13.0,1936.0,384.0,1158.0,367.0,2.7500,INLAND,5.275204,3.155313
...,...,...,...,...,...,...,...,...,...,...,...,...
2595,13229,-121.84,36.25,20.0,958.0,245.0,590.0,189.0,2.6094,NEAR OCEAN,5.068783,3.121693
2596,2085,-122.09,37.09,46.0,695.0,136.0,408.0,148.0,3.9408,NEAR OCEAN,4.695946,2.756757
2597,2795,-122.34,37.59,44.0,1395.0,269.0,736.0,288.0,5.6206,NEAR OCEAN,4.843750,2.555556
2598,5931,-122.34,37.57,39.0,2647.0,616.0,1254.0,555.0,4.2407,NEAR OCEAN,4.769369,2.259459


In [8]:
# laod model 
import pickle
with open('Final_Model_Capstone.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
y_pred = loaded_model.predict(new_test_data)
y_pred

array([138732.17, 189606.86], dtype=float32)

In [9]:
# prediction data observation
new_test_cloud = new_test_data.copy()
new_test_cloud['Prediction'] = y_pred
new_test_cloud

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,population_per_household,Prediction
0,-118.31,33.95,43.0,1823.0,358.0,1065.0,342.0,3.2708,<1H OCEAN,5.330409,3.114035,138732.171875
1,-122.30,37.90,30.0,1772.0,471.0,880.0,437.0,2.2672,NEAR BAY,4.054920,2.013730,189606.859375


In [10]:
# predict dataset
data_pred = loaded_model.predict(df)
data_pred[:10]

array([109672.1  ,  99085.88 , 102510.125, 116590.914,  89351.67 ,
       127467.914, 156428.58 , 132775.8  , 132723.25 ,  86946.71 ],
      dtype=float32)

In [11]:
# make a new column
df['Prediction'] =data_pred

In [12]:
df

,int64_field_0,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,population_per_household,Prediction
0,1656,-120.00,38.92,26.0,529.0,116.0,191.0,83.0,3.5000,INLAND,6.373494,2.301205,109672.101562
1,2057,-116.25,33.81,24.0,880.0,187.0,507.0,169.0,3.4583,INLAND,5.207101,3.000000,99085.882812
2,9295,-116.50,33.69,20.0,4810.0,1074.0,1304.0,740.0,2.2500,INLAND,6.500000,1.762162,102510.125000
3,7200,-120.50,38.87,10.0,81.0,41.0,55.0,16.0,4.9583,INLAND,5.062500,3.437500,116590.914062
4,9731,-120.50,37.32,13.0,1936.0,384.0,1158.0,367.0,2.7500,INLAND,5.275204,3.155313,89351.671875
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595,13229,-121.84,36.25,20.0,958.0,245.0,590.0,189.0,2.6094,NEAR OCEAN,5.068783,3.121693,172022.875000
2596,2085,-122.09,37.09,46.0,695.0,136.0,408.0,148.0,3.9408,NEAR OCEAN,4.695946,2.756757,233767.562500
2597,2795,-122.34,37.59,44.0,1395.0,269.0,736.0,288.0,5.6206,NEAR OCEAN,4.843750,2.555556,351012.656250
2598,5931,-122.34,37.57,39.0,2647.0,616.0,1254.0,555.0,4.2407,NEAR OCEAN,4.769369,2.259459,359052.031250


In [13]:
# save data prediction to biquery
from google.cloud import bigquery
table_id = 'unseen_house_clv_predicted'

client = bigquery.Client()

table_full_id = f"{client.project}.{dataset_id}.{table_id}"

df.column = ['_'.join(i.split(' ')) for i in df.columns]

job = client.load_table_from_dataframe(df, table_full_id)

job.result()
print(f"Loaded {job.output_rows} rows into{table_full_id}")

/tmp/ipykernel_6200/2191010261.py:9: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.column = ['_'.join(i.split(' ')) for i in df.columns]


Loaded 2600 rows intodti-ds.gian_dataset_023.unseen_house_clv_predicted
